In [48]:
import pandas as pd
import ipywidgets as widgets
from IPython.display import display, clear_output

# Cargar preguntas desde tu archivo Excel

df_preguntas  = pd.read_excel("Plantilla_Cuestionario_AWS.xlsx", sheet_name="Cuestionario")

preguntas = df_preguntas.to_dict(orient="records")
resultados = []

# Encabezado tipo banner
header_html = widgets.HTML(value="""
<div style='
    width: 100%;
    max-width: 1200px;
    padding: 40px;
    border-radius: 20px;
    background: linear-gradient(90deg, #0066cc, #3399ff);
    color: white;
    margin: 30px auto 20px auto;
    box-shadow: 0 8px 16px rgba(0,0,0,0.1);
    text-align: center;
'>
    <h1 style='margin: 0; font-size: 32px;'>Simulacro AWS Cloud Practitioner</h1>
    <p style='font-size: 18px; margin-top: 10px;'>Responde todas las preguntas y obtén tu puntaje final</p>
</div>
""")
display(header_html)

# Barra de progreso
barra_progreso = widgets.IntProgress(
    value=0,
    min=0,
    max=len(preguntas),
    description='Progreso:',
    bar_style='info',
    orientation='horizontal',
    layout=widgets.Layout(width='100%', max_width='1200px', margin='0 auto 30px auto')
)
display(barra_progreso)

# Contenedor global para las preguntas
contenedor_global = widgets.VBox(layout=widgets.Layout(width='100%', max_width='1200px', margin='0 auto'))
display(contenedor_global)

def mostrar_pregunta(i=0):
    barra_progreso.value = i
    contenedor_global.children = []

    if i >= len(preguntas):
        mostrar_resultado()
        return

    pregunta = preguntas[i]

    estilo_card = f"""
    <div style='
        border: 1px solid #ddd;
        border-radius: 12px;
        padding: 24px;
        background-color: #f9f9f9;
        margin-bottom: 20px;
        box-shadow: 0 2px 6px rgba(0,0,0,0.05);
    '>
        <h4 style='color:#222; margin: 0 0 10px;'>Pregunta {i+1}</h4>
        <p style='font-size: 18px; margin: 0;'>{pregunta['Pregunta']}</p>
    </div>
    """
    enunciado = widgets.HTML(value=estilo_card)

    opciones = [
        pregunta["Alternativa A"],
        pregunta["Alternativa B"],
        pregunta["Alternativa C"],
        pregunta["Alternativa D"]
    ]

    radio = widgets.RadioButtons(
    options=opciones,
    layout=widgets.Layout(width='auto'),
    style={'description_width': '0px'}
)

    boton_responder = widgets.Button(description="Responder", button_style='info')
    salida = widgets.Output()

    def al_hacer_click(b):
        respuesta_usuario = radio.value
        correcta = pregunta["Respuesta Correcta"]
        correcto = respuesta_usuario == correcta
        resultados.append((pregunta["Pregunta"], respuesta_usuario, correcta, correcto))

        with salida:
            clear_output()
            if correcto:
                print("✅ ¡Correcto!")
            else:
                print(f"❌ Incorrecto. La respuesta correcta es: {correcta}")

        boton_siguiente = widgets.Button(description="Siguiente", button_style='success')
        boton_siguiente.on_click(lambda x: mostrar_pregunta(i + 1))
        fila_boton = widgets.HBox([boton_siguiente], layout=widgets.Layout(justify_content='flex-end'))
        contenedor_global.children = [enunciado, radio, boton_responder, salida, fila_boton]

    boton_responder.on_click(al_hacer_click)
    contenedor_global.children = [enunciado, radio, boton_responder, salida]

def mostrar_resultado():
    clear_output()
    total = len(resultados)
    correctas = sum(1 for r in resultados if r[3])
    porcentaje = int((correctas / total) * 100)

    estilo = 'success' if porcentaje >= 70 else 'warning' if porcentaje >= 40 else 'danger'

    resumen = widgets.HTML(value=f"""
    <div style='
        max-width: 800px;
        margin: 40px auto;
        padding: 30px;
        border-radius: 15px;
        background: #f1f1f1;
        box-shadow: 0 4px 10px rgba(0,0,0,0.05);
        text-align: center;
    '>
        <h2 style='margin-top: 0;'>🎯 Resultado Final</h2>
        <p style='font-size: 20px;'>Respondiste <strong>{correctas}/{total}</strong> correctamente (<strong>{porcentaje}%</strong>)</p>
    </div>
    """)

    barra_final = widgets.IntProgress(
        value=porcentaje,
        min=0,
        max=100,
        description='Tu Puntaje:',
        bar_style=estilo,
        orientation='horizontal',
        layout=widgets.Layout(width='100%', max_width='800px', margin='0 auto 30px auto')
    )

    display(resumen, barra_final)

    for idx, (preg, resp, corr, ok) in enumerate(resultados):
        estado = "✅" if ok else "❌"
        print(f"{estado} {idx+1}. {preg}\n   Tu respuesta: {resp}\n   Correcta: {corr}\n")

# Iniciar cuestionario
mostrar_pregunta()


HTML(value="\n<div style='\n    width: 100%;\n    max-width: 1200px;\n    padding: 40px;\n    border-radius: 2…

IntProgress(value=0, bar_style='info', description='Progreso:', layout=Layout(margin='0 auto 30px auto', max_w…

VBox(layout=Layout(margin='0 auto', max_width='1200px', width='100%'))